In [1]:
import numpy as np
import pandas as pd
import torch
from tqdm.notebook import tqdm,trange
import torch.nn as nn
from random import shuffle


In [3]:
df=pd.read_csv('100計數_地段率1115.csv',encoding='UTF-8')

In [12]:
c=0
print(np.isnan(df.iloc[0]['平均公告現值']))



True


In [16]:
data=np.zeros((141,132,32))
label=np.zeros((141,132,1))
for x in range(len(data)):
    for y in range(len(data[0])):
        id=141*y+x
        datadict={
            0:'距最近火車站距離',
            1:'活動中心',
            2:'郵局',
            3:'公車站',
            4:'加油站',
            5:'停車場',
            6:'腳踏車',
            7:'寺廟',
            8:'大賣場',
            9:'服飾',
            10:'便利商店',
            11:'超市美妝',
            12:'電子賣場',
            13:'銀行',
            14:'ATM',
            15:'公園',
            16:'高中',
            17:'國小',
            18:'國中',
            19:'補習班',
            20:'食物',
            21:'診所',
            22:'禮品百貨',
            23:'醫院',
            24:'嫌惡設施_危險',
            25:'嫌惡設施_殯葬',
            26:'嫌惡設施_髒亂',
            27:'大學',
            28:'觀光景點',
            29:'平均公告現值',
            30:'加權容積率',
            31:'加權建蔽率',
            
        }
        label[x][y][0]=df.iloc[id]['平均地段率']
        for i in range(32):
            data[x][y][i]=df.iloc[id][datadict[i]]
            if  data[x][y][i]>0:
                temp=1
            else:
                data[x][y][i]=0


In [17]:


validnum=len(df[df['平均地段率']>0.1])
print(validnum)
validdata1=np.zeros((len(df[df['平均地段率']>0.1]),21,11,11))
validdata2=np.zeros((len(df[df['平均地段率']>0.1]),7,21,21))
validdata3=np.zeros((len(df[df['平均地段率']>0.1]),4))

validlabel=np.zeros((len(df[df['平均地段率']>0.1]),1))
counter=0
for y in trange(len(data[0])):
    for x in range(len(data)):
        if label[x][y][0]>0.5:
            for i in range(1,22):
                tempx=x-5
                tempy=y-5
                for a in range(11):
                    for b in range(11):
                        try:
                            validdata1[counter][i][a][b]+=data[tempx+a][tempy+b][i]
                        except:
                            continue
            for i in range(22,29):
                tempx=x-10
                tempy=y-10
                for a in range(21):
                    for b in range(21):
                        try:
                            validdata1[counter][i][a][b]+=data[tempx+a][tempy+b][i]
                        except:
                            continue
            
            validlabel[counter][0]=label[x][y][0]-120
            validdata3[counter][0]=data[x][y][0]
            validdata3[counter][1]=data[x][y][29]
            validdata3[counter][2]=data[x][y][30]
            validdata3[counter][3]=data[x][y][31]



            

4311


  0%|          | 0/132 [00:00<?, ?it/s]

In [18]:
from torch.utils.data import Dataset, DataLoader
class ExampleDataset(Dataset):

    # data loading
    def __init__(self,tr1,tr2,tr3,target, length):
        
        self.x1 = tr1.astype(np.float32)
        self.x2 = tr2.astype(np.float32)
        self.x3 = tr3.astype(np.float32)
        self.y = target.astype(np.float32)
        self.n_samples = length

    # working for indexing
    def __getitem__(self, index):
        
        return self.x1[index],self.x2[index],self.x3[index], self.y[index]

    # return the length of our dataset
    def __len__(self):
        
        return self.n_samples

In [24]:
k=[]

for i in range(len(validdata3)):
    k.append([validdata1[i],validdata2[i],validdata3[i],validlabel[i]])


r=np.random.permutation(k)
for i in range(len(validdata3)):
    validdata1[i]=k[i][0]
    validdata2[i]=k[i][1]
    validdata3[i]=k[i][2]
    validlabel[i]=k[i][3]
tr0_set=ExampleDataset(validdata1[0:2200],validdata2[0:2200],validdata3[0:2200],validlabel[0:2200],2200)
tr0_val=ExampleDataset(validdata1[2200:],validdata2[2200:],validdata3[2200:],validlabel[2200:],len(validdata3)-2200)
#
dataloader = DataLoader(tr0_set, batch_size=30, shuffle=True)
dataloader_val = DataLoader(tr0_val, batch_size=1, shuffle=False)

C:\Users\yehsanity\AppData\Local\Temp\ipykernel_24236\4285765929.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  r=np.random.permutation(k)


In [21]:
import torch.nn as nn
class NeuralNet(nn.Module):
    ''' A simple fully-connected deep neural network '''
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.cv2_1=nn.Conv2d(21, 64, 3, stride=1)
        self.cv2_2=nn.Conv2d(64, 256, 5, stride=1)
        self.cv2_3=nn.Conv2d(7, 32, 5, stride=1)
        self.cv2_4=nn.Conv2d(32, 64, 10, stride=1)

        self.linear1 = nn.Linear(4, 8)
        self.linear2 = nn.Linear(8, 8)

        self.layer1 = nn.Linear(5*5*256+8*8*64+8, 128)
        self.layer2 = nn.Linear(128, 32)
        self.out = nn.Linear(32, 1) 

        self.ba1=nn.BatchNorm3d(21)
        self.ba2=nn.BatchNorm3d(7)
        self.ba3=nn.BatchNorm1d(2)

        self.act_fn1 = nn.ReLU()
        self.act_fn2 = nn.ReLU()
        self.act_fn3 = nn.ReLU()
        self.act_fn4 = nn.ReLU()
        self.act_fn5 = nn.ReLU()
        # Define your neural network here
        # TODO: How to modify this model to achieve better performance?
        # self.net = nn.Sequential(
        #     nn.Conv2d(28, 64, 3, stride=1),
        #     nn.ReLU(),
        #     nn.Conv2d(64, 256, 5, stride=1),
        #     nn.ReLU(),
        #     nn.Flatten(),
        #     nn.Linear(5*5*256, 128),
        #     nn.ReLU(),
        #     nn.Linear(128, 32),
        #     nn.ReLU(),
        #     nn.Linear(32, 1) 
        # )

        # Mean squared error loss
        self.criterion = nn.MSELoss(reduction='mean')

    def forward(self, x1,x2,x3):
        ''' Given input of size (batch_size x input_dim), compute output of the network '''
        x1=self.cv2_1(x1)
        x1=self.act_fn1(x1)
        x1=self.cv2_2(x1)
        x1=nn.Flatten()(x1)
        x2=self.cv2_3(x2)
        x2=self.act_fn2(x2)
        x2=self.cv2_4(x2)
        x2=nn.Flatten()(x2)
        x3=self.linear1(x3)
        x3=self.act_fn3(x3)
        x3=self.linear2(x3)
        x = torch.cat((x1,x2,x3), dim=1)
        x = self.layer1(x)
        x = self.act_fn4(x)
        x = self.layer2(x)
        x = self.act_fn5(x)
        out=self.out(x)
        
        return out

    def cal_loss(self, pred, target):
        ''' Calculate loss '''
        # TODO: you may implement L1/L2 regularization here
        lamb = 0.0003
        if lamb :
          l2_reg = torch.tensor(0.).to('cuda')
          for param in self.parameters() :
            l2_reg += torch.norm(param)
          return self.criterion(pred, target) + l2_reg*lamb
        else : 
          return self.criterion(pred, target)

In [26]:
import torch.optim as optim

def train(tr_set, val_set, model, device):
    ''' DNN training '''
    criterion = nn.MSELoss()
    n_epochs = 50  # Maximum number of epochs
    best_loss = 0.0
    # Setup optimizer
    optimizer = optim.Adam(model.parameters(), lr=1E-4,weight_decay=0.00002)   

    min_mse = 1000.
    loss_record = {'train': [], 'dev': []}      # for recording training loss
    early_stop_cnt = 0
    epoch = 0
    best_loss = 100000.0
    while epoch < n_epochs:
        train_loss = 0.0
        val_acc = 0.0
        val_loss = 0.0
        model.train()                           # set model to training mode
        for x1,x2,x3, y in tr_set:                     # iterate through the dataloader
            optimizer.zero_grad()               # set gradient to zero
            x1,x2,x3, labels = x1.to(device),x2.to(device),x3.to(device), y.to(device)   # move data to device (cpu/cuda)
            outputs = model(x1,x2,x3)                     # forward pass (compute output)
            mse_loss = criterion(outputs, labels)  # compute loss
            _, train_pred = torch.max(outputs, 1)
            mse_loss.backward()                 # compute gradient (backpropagation)
            optimizer.step()                    # update model with optimizer
            train_loss += mse_loss.item()

        # After each epoch, test your model on the validation (development) set.
        if len(val_set) > 0:
            model.eval() # set the model to evaluation mode
            with torch.no_grad():
                for i, data in enumerate(val_set):
                    x1,x2,x3, labels = data
                    x1,x2,x3, labels = x1.to(device),x2.to(device),x3.to(device), labels.to(device)
                    outputs = model(x1,x2,x3)
                    batch_loss = criterion(outputs, labels) 
                    _, val_pred = torch.max(outputs, 1) 
                    an= torch.max(labels) 
                    val_loss += batch_loss.item()

                print('[{:03d}/{:03d}]  Loss: {:3.6f} | loss: {:3.6f}'.format(
                    epoch + 1, n_epochs, train_loss/len(tr_set),  val_loss/len(val_set)
                ))

                # if the model improves, save a checkpoint at this epoch
                if val_loss < best_loss:
                    best_loss = val_loss
                    torch.save(model, "D:\程式\google map api\model_cnn_muti2.pth")
                    print('saving model with loss {:.3f}'.format(best_loss/len(val_set)))
        else:
            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
                epoch + 1, n_epochs, train_loss/len(tr_set), train_loss/len(tr_set)
            ))
        epoch += 1
    # if not validating, save the last epoch
    if len(val_set) == 0:
        torch.save(model, 'model.pth')
        print('saving model at last epoch')

    return min_mse, loss_record

In [27]:

model = NeuralNet().to('cuda')  # Construct model and move to device
model_loss, model_loss_record = train(dataloader, dataloader_val, model, 'cuda')




[001/050]  Loss: 6.478383 | loss: 0.000129
saving model with loss 0.000
[002/050]  Loss: 2.206494 | loss: 0.000001
saving model with loss 0.000
[003/050]  Loss: 5.315324 | loss: 0.000005
[004/050]  Loss: 0.577614 | loss: 0.000001
saving model with loss 0.000
[005/050]  Loss: 0.280075 | loss: 0.000043
[006/050]  Loss: 0.023174 | loss: 0.000000
saving model with loss 0.000
[007/050]  Loss: 0.105275 | loss: 0.000007
[008/050]  Loss: 0.164901 | loss: 0.000046
[009/050]  Loss: 0.067839 | loss: 0.000000
[010/050]  Loss: 0.000006 | loss: 0.000000
saving model with loss 0.000
[011/050]  Loss: 0.000002 | loss: 0.000000
[012/050]  Loss: 0.000001 | loss: 0.000000
[013/050]  Loss: 0.000000 | loss: 0.000000
[014/050]  Loss: 0.000001 | loss: 0.000000
[015/050]  Loss: 0.000010 | loss: 0.000000
[016/050]  Loss: 0.000012 | loss: 0.000000
[017/050]  Loss: 0.000017 | loss: 0.000000
[018/050]  Loss: 0.000034 | loss: 0.000000
saving model with loss 0.000
[019/050]  Loss: 0.000081 | loss: 0.000000
saving mo

In [29]:

model=torch.load("model_muti1116.pth")
def test(tt_set, model, device):
    model.eval()                                # set model to evalutation mode
    preds = []
    for x1,x2,x3, y in tt_set:                            # iterate through the dataloader
        x1,x2,x3 = x1.to(device),x2.to(device),x3.to(device)                       # move data to device (cpu/cuda)
        with torch.no_grad():                   # disable gradient calculation
            pred = model(x1,x2,x3)                     # forward pass (compute output)
            preds.append(pred.detach().cpu())   # collect prediction
    preds = torch.cat(preds, dim=0).numpy()     # concatenate all predictions and convert to a numpy array
    return preds

count=0
prd_err=[]
prd_set=ExampleDataset(validdata1,validdata2,validdata3,validlabel,len(validlabel))
tt_set = DataLoader(prd_set, batch_size=1, shuffle=False)
preds = test(tt_set, model, 'cuda')  # predict COVID-19 cases with your model
for i in range(len(preds)):
    if preds[i] - validlabel[i] > 10 or preds[i] - validlabel[i] < -10:
        print("實際值:",validlabel[i],"預測結果:",preds[i],"id:",i)
        count+=1
        prd_err.append(i)
print(len(preds))

print("預測錯誤數量:",count)
prd_err=pd.DataFrame(prd_err)
prd_err.to_csv("prd_err_cnn.csv",index=False)



實際值: [10.] 預測結果: [157.3305] id: 0
4311
預測錯誤數量: 1


In [32]:
#real data(all)


realdata1=np.zeros((len(df),21,11,11))
realdata2=np.zeros((len(df),7,21,21))
realdata3=np.zeros((len(df),4))

counter=0
for y in trange(len(data[0])):
    for x in range(len(data)):
        for i in range(1,22):
            tempx=x-5
            tempy=y-5
            for a in range(11):
                for b in range(11):
                    try:
                        realdata1[counter][i][a][b]+=data[tempx+a][tempy+b][i]
                    except:
                        continue
        for i in range(22,29):
            tempx=x-10
            tempy=y-10
            for a in range(21):
                for b in range(21):
                    try:
                        realdata1[counter][i][a][b]+=data[tempx+a][tempy+b][i]
                    except:
                        continue

        
        realdata3[counter][0]=data[x][y][0]
        realdata3[counter][1]=data[x][y][29]
        realdata3[counter][2]=data[x][y][30]
        realdata3[counter][3]=data[x][y][31]
        



  0%|          | 0/132 [00:00<?, ?it/s]

In [33]:

prd_set2=ExampleDataset(realdata1,realdata2,realdata3,realdata3,len(realdata3))
tt_set = DataLoader(prd_set2, batch_size=1, shuffle=False)
preds = test(tt_set, model, 'cuda')  # predict COVID-19 cases with your model

In [34]:
import math
err=[]
pred=[]
for i in range(len(preds)):
    ##err.append(abs(preds[i][0]-reallabel[i][0]))
    pred.append(preds[i][0]+120)


In [35]:
print(len(pred))

18612


In [ ]:

df['預測值']=pred
df.to_csv('result100all_cnn_muti_120.csv')

In [ ]:
import decimal
import json
with open('100計數_地段率1115.geojson', newline='',encoding="utf-8") as jsonfile:
    data = json.load(jsonfile)

for i in range(len(preds)):
    data['features'][i]['properties']['預測值']=pred[i]

print(data['features'][0]['properties'])
with open('100計數_地段率1115_pred.geojson', 'w',encoding="utf-8") as json_obj:
    json.dump(data, json_obj, ensure_ascii = False, indent=4)




with open('100計數_地段率1115_pred.geojson', newline='',encoding="utf-8") as jsonfile2:
    data2 = json.load(jsonfile2)
print(data2['features'][0]['properties'])

{'id': 1.0, 'left': 13373240.3256, 'top': 2635136.2672999995, 'right': 13373340.325599998, 'bottom': 2635036.2673, '活動中心': 0.0, '郵局': 0.0, '公車站': 0.0, '火車站': 0.0, '加油站': 0.0, '停車場': 0.0, '腳踏車': 0.0, '寺廟': 0.0, '大賣場': 0.0, '服飾': 0.0, '便利商店': 0.0, '超市美妝': 0.0, '電子賣場': 0.0, '銀行': 0.0, 'ATM': 0.0, '大學': 0.0, '高中': 0.0, '國小': 0.0, '國中': 0.0, '補習班': 0.0, '嫌惡設施_危險': 0.0, '嫌惡設施_殯葬': 0.0, '嫌惡設施_髒亂': 0.0, '食物': 0.0, '診所': 0.0, '醫院': 0.0, '禮品百貨': 0.0, '觀光景點': 0.0, '房屋稅數量': 0.0, '地段率總和': 0.0, '折舊年數總和': 0.0, '平均地段率': None, '平均折舊年數': None, 'EPSG3857_X': 13373290.325599998, 'EPSG3857_Y': 2635086.2672999995, '距最近火車站距離': 8583.1767164421, '公園': 0.0, '土地單價': None, '預測值': 108.66443061828613}
{'id': 1.0, 'left': 13373240.3256, 'top': 2635136.2672999995, 'right': 13373340.325599998, 'bottom': 2635036.2673, '活動中心': 0.0, '郵局': 0.0, '公車站': 0.0, '火車站': 0.0, '加油站': 0.0, '停車場': 0.0, '腳踏車': 0.0, '寺廟': 0.0, '大賣場': 0.0, '服飾': 0.0, '便利商店': 0.0, '超市美妝': 0.0, '電子賣場': 0.0, '銀行': 0.0, 'ATM': 0.0, '大學': 0.0, '高中': 0.0, '國